In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade transformers accelerate bitsandbytes safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 132.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
!pip install git+https://github.com/salaniz/pycocoevalcap

  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-2h2es2qh
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-2h2es2qh
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=46caba41d3a2be0e54882a628354756fe4c7c14de955c4a2d29edc63aa0d53d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-9u6kd5qo/wheels/03/ce/0b/3d3fdeecb09b4f4ebcfb3ff28d27a9f5b3c1a7b73897ad122d
Successfully built pycocoevalcap


In [ ]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 157.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
from collections import defaultdict
from datasets import load_dataset

# MSR-VTT test_1k 불러오기
ds = load_dataset("friedrichor/MSR-VTT", "test_1k")

# video_id 기준으로 caption들을 묶기
caption_dict = defaultdict(lambda: {"captions": []})

for item in ds["test"]:
    vid = str(item["video_id"])
    cap = item["caption"]
    caption_dict[vid]["captions"].append(cap)

# JSON 저장
save_path = "/content/drive/MyDrive/video/MSRVTT/captions_test_1k.json"
with open(save_path, "w") as f:
    json.dump(caption_dict, f, indent=2)

print(f"✅ captions_test.json 저장 완료: {save_path}")
print(f"총 비디오 수: {len(caption_dict)}")


✅ captions_test.json 저장 완료: /content/drive/MyDrive/video/MSRVTT/captions_test_1k.json
총 비디오 수: 1000


In [3]:
from transformers import CLIPProcessor, CLIPModel
import torch, os, json, time
from PIL import Image
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)
model.eval()

# 프레임 리사이즈
def _resize_one(args):
    in_path, out_path, size = args
    try:
        img = Image.open(in_path).convert("RGB")
        img = img.resize(size)
        img.save(out_path)
    except Exception:
        pass

def resize_frames_fast(input_dir, output_dir, size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    tasks = []
    for vid in sorted(os.listdir(input_dir)):
        in_vid = os.path.join(input_dir, vid)
        out_vid = os.path.join(output_dir, vid)
        os.makedirs(out_vid, exist_ok=True)
        for f in os.listdir(in_vid):
            if f.endswith(".jpg"):
                tasks.append((os.path.join(in_vid, f),
                              os.path.join(out_vid, f),
                              size))
    print(f"{len(tasks)} frames resizing using {cpu_count()} workers...")
    with Pool(cpu_count()) as p:
        list(tqdm(p.imap_unordered(_resize_one, tasks), total=len(tasks)))
    print("모든 프레임 리사이즈 완료 (224x224).")

# 비디오 임베딩 추출
def extract_video_embedding(frame_dir, num_frames=8, batch_size=8):
    frame_files = sorted(os.listdir(frame_dir))[:num_frames]
    images = [Image.open(os.path.join(frame_dir, f)).convert("RGB") for f in frame_files]

    embeds = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt", padding=True).to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                feats = model.get_image_features(**inputs)
            embeds.append(feats)
    embeds = torch.cat(embeds, dim=0)
    return embeds.mean(dim=0).cpu()

# 텍스트 임베딩 추출 (mean pooling)
def get_text_embeddings(caption_dict, video_ids):
    all_embeds = []
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
        for vid in tqdm(video_ids, desc="Extracting text embeddings (mean pooled)"):
            caps = caption_dict[vid]["captions"]
            inputs = processor(text=caps, return_tensors="pt", padding=True, truncation=True).to(device)
            feats = model.get_text_features(**inputs)
            feats = torch.nn.functional.normalize(feats, dim=-1)
            mean_feat = feats.mean(dim=0, keepdim=True)
            all_embeds.append(mean_feat.cpu())
    return torch.cat(all_embeds, dim=0)

# Recall@K 계산
def recall_at_k(sim, k=1):
    ranks = sim.argsort(dim=1, descending=True)
    topk = ranks[:, :k]
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    hits = (topk == correct).any(dim=1).float()
    return hits.mean().item()

# 전체 실행 파이프라인
start_total = time.time()

original_root = "/content/drive/MyDrive/video/Test_1kA_frames8"
resized_root = "/content/drive/MyDrive/video/Test_1kA_frames8_224"
resize_frames_fast(original_root, resized_root)

video_root = resized_root
video_ids = sorted(os.listdir(video_root))
print(f"총 비디오 개수: {len(video_ids)}개")

with open("/content/drive/MyDrive/video/MSRVTT/captions_test_1k.json", "r") as f:
    caption_dict = json.load(f)

# 비디오 임베딩 추출
video_embeds = []
for vid in tqdm(video_ids, desc="Extracting video embeddings (8 frames)"):
    try:
        video_embeds.append(extract_video_embedding(os.path.join(video_root, vid)))
    except Exception as e:
        print(f"{vid} 처리 중 오류 발생: {e}")
video_mat = torch.stack(video_embeds)
print(f"비디오 임베딩 완료 ({len(video_embeds)}개)")

# 텍스트 임베딩 추출
text_mat = get_text_embeddings(caption_dict, video_ids)
print(f"텍스트 임베딩 완료 ({len(text_mat)}개)")

# 저장
save_dir = "/content/drive/MyDrive/video/MSRVTT/"
os.makedirs(save_dir, exist_ok=True)
torch.save(video_mat, os.path.join(save_dir, "clip_video_embeds_8f.pt"))
torch.save(text_mat, os.path.join(save_dir, "clip_text_embeds_8f.pt"))
print("임베딩 저장 완료.")

# 유사도 계산 및 리트리벌 평가
video_mat = torch.nn.functional.normalize(video_mat.to(torch.float32), dim=-1)
text_mat = torch.nn.functional.normalize(text_mat.to(torch.float32), dim=-1)
sim = video_mat @ text_mat.T

print("\nRetrieval 성능 (8frame)")
for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k(sim, k):.4f}")

end_total = time.time()
print(f"\n전체 소요 시간: {(end_total - start_total)/60:.2f}분")


8000 frames resizing using 12 workers...
100%|██████████| 8000/8000 [08:21<00:00, 15.95it/s]
모든 프레임 리사이즈 완료 (224x224).
총 비디오 개수: 1000개
Extracting video embeddings (8 frames): 100%|██████████| 1000/1000 [00:43<00:00, 22.89it/s]
비디오 임베딩 완료 (1000개)
Extracting text embeddings (mean pooled): 100%|██████████| 1000/1000 [00:10<00:00, 98.19it/s]
텍스트 임베딩 완료 (1000개)
임베딩 저장 완료.

Retrieval 성능 (8frame)
Recall@1: 0.2640
Recall@5: 0.5100
Recall@10: 0.6290

전체 소요 시간: 9.2분


In [6]:
# 모델 로드 (FP16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)
model.eval()

#  병렬 프레임 리사이즈
def _resize_one(args):
    in_path, out_path, size = args
    try:
        img = Image.open(in_path).convert("RGB")
        img = img.resize(size)
        img.save(out_path)
    except Exception:
        pass

def resize_frames_fast(input_dir, output_dir, size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    tasks = []
    for vid in sorted(os.listdir(input_dir)):
        in_vid = os.path.join(input_dir, vid)
        out_vid = os.path.join(output_dir, vid)
        os.makedirs(out_vid, exist_ok=True)
        for f in os.listdir(in_vid):
            if f.endswith(".jpg"):
                tasks.append((os.path.join(in_vid, f),
                              os.path.join(out_vid, f),
                              size))
    print(f"{len(tasks)} frames resizing using {cpu_count()} workers...")
    with Pool(cpu_count()) as p:
        list(tqdm(p.imap_unordered(_resize_one, tasks), total=len(tasks)))
    print(" 모든 프레임 리사이즈 완료 (224x224).")

# 16프레임 버전 경로
original_root = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_frames16"
resized_root = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_frames16_224"
resize_frames_fast(original_root, resized_root)

# 비디오 임베딩 추출 (16프레임)
def extract_video_embedding(frame_dir, num_frames=16, batch_size=8):
    frame_files = sorted(os.listdir(frame_dir))[:num_frames]
    images = [Image.open(os.path.join(frame_dir, f)).convert("RGB") for f in frame_files]
    embeds = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt", padding=True).to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                feats = model.get_image_features(**inputs)
            embeds.append(feats)
    embeds = torch.cat(embeds, dim=0)
    return embeds.mean(dim=0).cpu()

# 텍스트 임베딩 (mean pooling)
def get_text_embeddings(caption_dict, video_ids):
    all_embeds = []
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
        for vid in tqdm(video_ids, desc="Extracting text embeddings (mean pooled)"):
            caps = caption_dict[vid]["captions"]
            inputs = processor(text=caps, return_tensors="pt", padding=True, truncation=True).to(device)
            feats = model.get_text_features(**inputs)
            feats = torch.nn.functional.normalize(feats, dim=-1)
            mean_feat = feats.mean(dim=0, keepdim=True)
            all_embeds.append(mean_feat.cpu())
    return torch.cat(all_embeds, dim=0)

# 비디오 및 캡션 로드
video_root = resized_root
video_ids = sorted(os.listdir(video_root))

with open("/content/drive/MyDrive/VIDEO_RETRIEVAL/captions_test_1k.json", "r") as f:
    caption_dict = json.load(f)

print(f"총 비디오 개수: {len(video_ids)}개")

# 비디오 임베딩 추출
start_time = time.time()
video_embeds = []
for vid in tqdm(video_ids, desc="Extracting video embeddings (16 frames)"):
    path = os.path.join(video_root, vid)
    try:
        video_embeds.append(extract_video_embedding(path))
    except Exception as e:
        print(f"{vid} 처리 중 오류 발생: {e}")

video_mat = torch.stack(video_embeds)
print(f"비디오 임베딩 완료 ({len(video_embeds)}개)")

# 텍스트 임베딩 추출
text_mat = get_text_embeddings(caption_dict, video_ids)

# 캐시 저장
save_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/"
os.makedirs(save_dir, exist_ok=True)
torch.save(video_mat, os.path.join(save_dir, "clip_video_embeds_16f.pt"))
torch.save(text_mat, os.path.join(save_dir, "clip_text_embeds_16f.pt"))
print("임베딩 저장 완료.")

# Recall@K 계산
video_mat = torch.nn.functional.normalize(video_mat, dim=-1)
text_mat = torch.nn.functional.normalize(text_mat, dim=-1)

video_mat = video_mat.to(torch.float32)
text_mat = text_mat.to(torch.float32)

sim = video_mat @ text_mat.T

def recall_at_k(sim, k=1):
    ranks = sim.argsort(dim=1, descending=True)
    topk = ranks[:, :k]
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    hits = (topk == correct).any(dim=1).float()
    return hits.mean().item()

print("\n Retrieval 성능 (16 frames)")
for k in [1,5,10]:
    print(f"Recall@{k}: {recall_at_k(sim, k):.4f}")

end_time = time.time()
print(f"\n 총 소요 시간: {(end_time - start_time)/60:.2f}분")

16000 frames resizing using 2 workers...
100%|██████████| 16000/16000 [1:34:15<00:00,  2.83it/s]
모든 프레임 리사이즈 완료 (224x224).
총 비디오 개수: 1000개
Extracting video embeddings (16 frames): 100%|██████████| 1000/1000 [01:26:37<00:00,  5.19s/it]
비디오 임베딩 완료 (1000개)
Extracting text embeddings (mean pooled): 100%|██████████| 1000/1000 [00:10<00:00, 98.27it/s]
텍스트 임베딩 완료 (1000개)
임베딩 저장 완료.

Retrieval 성능 (16 frames)
Recall@1: 0.2730
Recall@5: 0.5260
Recall@10: 0.6380

전체 소요 시간: 97.2분


In [7]:
from transformers import CLIPProcessor, CLIPModel
import torch, os
from PIL import Image
from tqdm import tqdm
import json
import time
from multiprocessing import Pool, cpu_count
import warnings

warnings.filterwarnings("ignore", message=".*padding.*")

# 모델 로드 (FP16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)
model.eval()

# 병렬 프레임 리사이즈
def _resize_one(args):
    in_path, out_path, size = args
    try:
        img = Image.open(in_path).convert("RGB")
        img = img.resize(size)
        img.save(out_path)
    except Exception:
        pass

def resize_frames_fast(input_dir, output_dir, size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    tasks = []
    for vid in sorted(os.listdir(input_dir)):
        in_vid = os.path.join(input_dir, vid)
        out_vid = os.path.join(output_dir, vid)
        os.makedirs(out_vid, exist_ok=True)
        for f in os.listdir(in_vid):
            if f.endswith(".jpg"):
                tasks.append((os.path.join(in_vid, f),
                              os.path.join(out_vid, f),
                              size))
    print(f" {len(tasks)} frames resizing using {cpu_count()} workers...")
    with Pool(cpu_count()) as p:
        list(tqdm(p.imap_unordered(_resize_one, tasks), total=len(tasks)))
    print("모든 프레임 리사이즈 완료 (224x224)")

# 32프레임 버전 경로
original_root = "/content/drive/MyDrive/video/MSRVTT/Test_1kA_frames32"
resized_root = "/content/drive/MyDrive/video/MSRVTT/Test_1kA_frames32_224"
resize_frames_fast(original_root, resized_root)

# 비디오 임베딩 추출 (32프레임)
def extract_video_embedding(frame_dir, num_frames=32, batch_size=8):
    frame_files = sorted(os.listdir(frame_dir))[:num_frames]
    images = [Image.open(os.path.join(frame_dir, f)).convert("RGB") for f in frame_files]
    embeds = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt").to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                feats = model.get_image_features(**inputs)
            embeds.append(feats)
    embeds = torch.cat(embeds, dim=0)
    return embeds.mean(dim=0).cpu()

# 텍스트 임베딩 (mean pooling)
def get_text_embeddings(caption_dict, video_ids):
    all_embeds = []
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
        for vid in tqdm(video_ids, desc="Extracting text embeddings (mean pooled)"):
            caps = caption_dict[vid]["captions"]
            inputs = processor(text=caps, return_tensors="pt", padding=True, truncation=True).to(device)
            feats = model.get_text_features(**inputs)
            feats = torch.nn.functional.normalize(feats, dim=-1)
            mean_feat = feats.mean(dim=0, keepdim=True)
            all_embeds.append(mean_feat.cpu())
    return torch.cat(all_embeds, dim=0)

# 비디오 및 캡션 로드
video_root = resized_root
video_ids = sorted(os.listdir(video_root))

with open("/content/drive/MyDrive/video/MSRVTT/captions_test_1k.json", "r") as f:
    caption_dict = json.load(f)

print(f"총 비디오 개수: {len(video_ids)}개")

# 비디오 임베딩 추출
start_time = time.time()
video_embeds = []
for vid in tqdm(video_ids, desc="Extracting video embeddings (32 frames)"):
    path = os.path.join(video_root, vid)
    try:
        video_embeds.append(extract_video_embedding(path))
    except Exception as e:
        print(f" {vid} 처리 중 오류 발생: {e}")

video_mat = torch.stack(video_embeds)
print(f" 비디오 임베딩 완료 ({len(video_embeds)}개)")

# 텍스트 임베딩 추출
text_mat = get_text_embeddings(caption_dict, video_ids)

# 캐시 저장
save_dir = "/content/drive/MyDrive/video/MSRVTT/"
os.makedirs(save_dir, exist_ok=True)
torch.save(video_mat, os.path.join(save_dir, "clip_video_embeds_32f.pt"))
torch.save(text_mat, os.path.join(save_dir, "clip_text_embeds_32f.pt"))

# Recall@K 계산
video_mat = torch.nn.functional.normalize(video_mat, dim=-1)
text_mat = torch.nn.functional.normalize(text_mat, dim=-1)

video_mat = video_mat.to(torch.float32)
text_mat = text_mat.to(torch.float32)

sim = video_mat @ text_mat.T

def recall_at_k(sim, k=1):
    ranks = sim.argsort(dim=1, descending=True)
    topk = ranks[:, :k]
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    hits = (topk == correct).any(dim=1).float()
    return hits.mean().item()

print("\n Retrieval 성능 (32 frames)")
for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k(sim, k):.4f}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
32000 frames resizing using 12 workers...
100%|██████████| 32000/32000 [03:26<00:00, 154.64it/s]
모든 프레임 리사이즈 완료 (224x224).
총 비디오 개수: 1000개
Extracting video embeddings (32 frames): 100%|██████████| 1000/1000 [02:58<00:00,  5.62it/s]
비디오 임베딩 완료 (1000개)
Extracting text embeddings (mean pooled): 100%|██████████| 1000/1000 [00:11<00:00, 86.13it/s]
캐시 저장 완료!

Retrieval 성능 (32 frames)
Recall@1: 0.2800
Recall@5: 0.5220
Recall@10: 0.6410

총 소요 시간: 3.16분


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# 원본 비디오 폴더
video_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA"

# 저장 경로 (랜덤 샘플링용)
save_dir_8  = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_random8"
save_dir_16 = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_random16"
save_dir_32 = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_random32"

for d in [save_dir_8, save_dir_16, save_dir_32]:
    os.makedirs(d, exist_ok=True)

def random_sample_indices(total_frames, target_frames):
    """비디오 길이에서 랜덤하게 프레임 인덱스 선택"""
    if total_frames <= target_frames:
        return np.arange(total_frames)
    return np.sort(np.random.choice(total_frames, target_frames, replace=False))

def extract_frames(video_path, save_root, target_frames):
    """비디오에서 랜덤 프레임 추출"""
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    save_path = os.path.join(save_root, video_name)
    os.makedirs(save_path, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    indices = random_sample_indices(total_frames, target_frames)

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    for i, idx in enumerate(indices):
        frame = frames[idx]
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        out_path = os.path.join(save_path, f"frame_{i}.jpg")
        cv2.imwrite(out_path, cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR))

# 모든 비디오 처리
videos = [f for f in os.listdir(video_dir) if f.endswith(('.mp4'))]

for v in tqdm(videos, desc="Extracting random-sampled frames"):
    video_path = os.path.join(video_dir, v)
    extract_frames(video_path, save_dir_8, 8)
    extract_frames(video_path, save_dir_16, 16)
    extract_frames(video_path, save_dir_32, 32)


Extracting random-sampled frames: 100%|██████████| 1000/1000 [32:02<00:00,  1.92s/it]

✅ 랜덤 샘플링 완료 — 8, 16, 32 프레임 버전 모두 생성됨.
폴더 구조 예시: Test_1kA_random16/videoXXXX/frame_0.jpg ...


In [ ]:
import os

# 랜덤 샘플링 결과 확인 코드
video_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA"

frames8_dir  = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_random8"
frames16_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_random16"
frames32_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Test_1kA_random32"

# 비디오 파일 목록
videos = [os.path.splitext(f)[0] for f in os.listdir(video_dir) if f.endswith(('.mp4'))]

missing_8, missing_16, missing_32 = [], [], []
wrong_8, wrong_16, wrong_32 = [], [], []

for v in videos:
    path8  = os.path.join(frames8_dir, v)
    path16 = os.path.join(frames16_dir, v)
    path32 = os.path.join(frames32_dir, v)

    # 8프레임
    if not os.path.exists(path8):
        missing_8.append(v)
    else:
        n8 = len([f for f in os.listdir(path8) if f.endswith(".jpg")])
        if n8 != 8:
            wrong_8.append((v, n8))

    # 16프레임
    if not os.path.exists(path16):
        missing_16.append(v)
    else:
        n16 = len([f for f in os.listdir(path16) if f.endswith(".jpg")])
        if n16 != 16:
            wrong_16.append((v, n16))

    # 32프레임
    if not os.path.exists(path32):
        missing_32.append(v)
    else:
        n32 = len([f for f in os.listdir(path32) if f.endswith(".jpg")])
        if n32 != 32:
            wrong_32.append((v, n32))

# 요약 출력
print(f"총 비디오 수: {len(videos)}")
print(f"8프레임 폴더 누락: {len(missing_8)}개")
print(f"16프레임 폴더 누락: {len(missing_16)}개")
print(f"32프레임 폴더 누락: {len(missing_32)}개")
print(f"8프레임 개수 오류: {len(wrong_8)}개")
print(f"16프레임 개수 오류: {len(wrong_16)}개")
print(f"32프레임 개수 오류: {len(wrong_32)}개\n")


총 비디오 수: 1000
8프레임 폴더 누락: 0개
16프레임 폴더 누락: 0개
32프레임 폴더 누락: 0개
8프레임 개수 오류: 0개
16프레임 개수 오류: 0개
32프레임 개수 오류: 0개



In [9]:
from transformers import CLIPProcessor, CLIPModel
import torch, os
from PIL import Image
from tqdm import tqdm
import json
import time
from multiprocessing import Pool, cpu_count
import warnings

warnings.filterwarnings("ignore", message=".*padding.*")

# 모델 로드 (FP16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)
model.eval()

# 병렬 프레임 리사이즈
def _resize_one(args):
    in_path, out_path, size = args
    try:
        img = Image.open(in_path).convert("RGB")
        img = img.resize(size)
        img.save(out_path)
    except Exception:
        pass

def resize_frames_fast(input_dir, output_dir, size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    tasks = []
    for vid in sorted(os.listdir(input_dir)):
        in_vid = os.path.join(input_dir, vid)
        out_vid = os.path.join(output_dir, vid)
        os.makedirs(out_vid, exist_ok=True)
        for f in os.listdir(in_vid):
            if f.endswith(".jpg"):
                tasks.append((os.path.join(in_vid, f),
                              os.path.join(out_vid, f),
                              size))
    print(f" {len(tasks)} frames resizing using {cpu_count()} workers...")
    with Pool(cpu_count()) as p:
        list(tqdm(p.imap_unordered(_resize_one, tasks), total=len(tasks)))
    print("모든 프레임 리사이즈 완료 (224x224)")

# 8프레임 랜덤 샘플링 버전 경로
base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL"
original_root = os.path.join(base_dir, "Test_1kA_random8")
resized_root = os.path.join(base_dir, "Test_1kA_random8_224")
resize_frames_fast(original_root, resized_root)

# 비디오 임베딩 추출 (8프레임)
def extract_video_embedding(frame_dir, num_frames=8, batch_size=8):
    frame_files = sorted(os.listdir(frame_dir))[:num_frames]
    images = [Image.open(os.path.join(frame_dir, f)).convert("RGB") for f in frame_files]
    embeds = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt").to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                feats = model.get_image_features(**inputs)
            embeds.append(feats)
    embeds = torch.cat(embeds, dim=0)
    return embeds.mean(dim=0).cpu()

# 텍스트 임베딩 (mean pooling)
def get_text_embeddings(caption_dict, video_ids):
    all_embeds = []
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
        for vid in tqdm(video_ids, desc="Extracting text embeddings (mean pooled)"):
            caps = caption_dict[vid]["captions"]
            inputs = processor(text=caps, return_tensors="pt", padding=True, truncation=True).to(device)
            feats = model.get_text_features(**inputs)
            feats = torch.nn.functional.normalize(feats, dim=-1)
            mean_feat = feats.mean(dim=0, keepdim=True)
            all_embeds.append(mean_feat.cpu())
    return torch.cat(all_embeds, dim=0)

# 비디오 및 캡션 로드
video_root = resized_root
video_ids = sorted(os.listdir(video_root))

with open(os.path.join(base_dir, "captions_test_1k.json"), "r") as f:
    caption_dict = json.load(f)

print(f"총 비디오 개수: {len(video_ids)}개")

# 비디오 임베딩 추출
start_time = time.time()
video_embeds = []
for vid in tqdm(video_ids, desc="Extracting video embeddings (8 random frames)"):
    path = os.path.join(video_root, vid)
    try:
        video_embeds.append(extract_video_embedding(path))
    except Exception as e:
        print(f"{vid} 처리 중 오류 발생: {e}")

video_mat = torch.stack(video_embeds)
print(f"비디오 임베딩 완료 ({len(video_embeds)}개)")

# 텍스트 임베딩 추출
text_mat = get_text_embeddings(caption_dict, video_ids)

# 캐시 저장
torch.save(video_mat, os.path.join(base_dir, "clip_video_embeds_random8f.pt"))
torch.save(text_mat, os.path.join(base_dir, "clip_text_embeds_random8f.pt"))

# Recall@K 계산
video_mat = torch.nn.functional.normalize(video_mat, dim=-1)
text_mat = torch.nn.functional.normalize(text_mat, dim=-1)

video_mat = video_mat.to(torch.float32)
text_mat = text_mat.to(torch.float32)

sim = video_mat @ text_mat.T

def recall_at_k(sim, k=1):
    ranks = sim.argsort(dim=1, descending=True)
    topk = ranks[:, :k]
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    hits = (topk == correct).any(dim=1).float()
    return hits.mean().item()

print("\n Retrieval 성능 (8 random frames)")
for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k(sim, k):.4f}")



7504 frames resizing using 2 workers...
100%|██████████| 7504/7504 [16:24<00:00,  7.62it/s]
모든 프레임 리사이즈 완료 (224x224).
총 비디오 개수: 1000개
Extracting video embeddings (8 random frames):   0%|          | 0/1000 [00:00<?, ?it/s]
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Extracting video embeddings (8 random frames): 100%|██████████| 1000/1000 [55:12<00:00,  3.31s/it]
비디오 임베딩 완료 (1000개)
Extracting text embeddings (mean pooled): 100%|██████████| 1000/1000 [00:10<00:00, 99.82it/s]
텍스트 임베딩 완료 (1000개)

Retrieval 성능 (8 random frames)
Recall@1: 0.2710
Recall@5: 0.5190
Recall@10: 0.6280

총 소요 시간: 65.4분


In [10]:

from transformers import CLIPProcessor, CLIPModel
import torch, os
from PIL import Image
from tqdm import tqdm
import json
import time
from multiprocessing import Pool, cpu_count
import warnings

warnings.filterwarnings("ignore", message=".*padding.*")

# 모델 로드 (FP16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)
model.eval()

# 병렬 프레임 리사이즈
def _resize_one(args):
    in_path, out_path, size = args
    try:
        img = Image.open(in_path).convert("RGB")
        img = img.resize(size)
        img.save(out_path)
    except Exception:
        pass

def resize_frames_fast(input_dir, output_dir, size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    tasks = []
    for vid in sorted(os.listdir(input_dir)):
        in_vid = os.path.join(input_dir, vid)
        out_vid = os.path.join(output_dir, vid)
        os.makedirs(out_vid, exist_ok=True)
        for f in os.listdir(in_vid):
            if f.endswith(".jpg"):
                tasks.append((os.path.join(in_vid, f),
                              os.path.join(out_vid, f),
                              size))
    print(f" {len(tasks)} frames resizing using {cpu_count()} workers...")
    with Pool(cpu_count()) as p:
        list(tqdm(p.imap_unordered(_resize_one, tasks), total=len(tasks)))
    print("모든 프레임 리사이즈 완료 (224x224).")

# 16프레임 랜덤 샘플링 버전 경로
base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL"
original_root = os.path.join(base_dir, "Test_1kA_random16")
resized_root = os.path.join(base_dir, "Test_1kA_random16_224")
resize_frames_fast(original_root, resized_root)

# 비디오 임베딩 추출 (16프레임)
def extract_video_embedding(frame_dir, num_frames=16, batch_size=8):
    frame_files = sorted(os.listdir(frame_dir))[:num_frames]
    images = [Image.open(os.path.join(frame_dir, f)).convert("RGB") for f in frame_files]
    embeds = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt").to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                feats = model.get_image_features(**inputs)
            embeds.append(feats)
    embeds = torch.cat(embeds, dim=0)
    return embeds.mean(dim=0).cpu()

# 텍스트 임베딩 (mean pooling)
def get_text_embeddings(caption_dict, video_ids):
    all_embeds = []
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
        for vid in tqdm(video_ids, desc="Extracting text embeddings (mean pooled)"):
            caps = caption_dict[vid]["captions"]
            inputs = processor(text=caps, return_tensors="pt", padding=True, truncation=True).to(device)
            feats = model.get_text_features(**inputs)
            feats = torch.nn.functional.normalize(feats, dim=-1)
            mean_feat = feats.mean(dim=0, keepdim=True)
            all_embeds.append(mean_feat.cpu())
    return torch.cat(all_embeds, dim=0)

# 비디오 및 캡션 로드
video_root = resized_root
video_ids = sorted(os.listdir(video_root))

with open(os.path.join(base_dir, "captions_test_1k.json"), "r") as f:
    caption_dict = json.load(f)

print(f"총 비디오 개수: {len(video_ids)}개")

# 비디오 임베딩 추출
start_time = time.time()
video_embeds = []
for vid in tqdm(video_ids, desc="Extracting video embeddings (16 random frames)"):
    path = os.path.join(video_root, vid)
    try:
        video_embeds.append(extract_video_embedding(path))
    except Exception as e:
        print(f"{vid} 처리 중 오류 발생: {e}")

video_mat = torch.stack(video_embeds)
print(f"비디오 임베딩 완료 ({len(video_embeds)}개)")

# 텍스트 임베딩 추출
text_mat = get_text_embeddings(caption_dict, video_ids)

# 캐시 저장
torch.save(video_mat, os.path.join(base_dir, "clip_video_embeds_random16f.pt"))
torch.save(text_mat, os.path.join(base_dir, "clip_text_embeds_random16f.pt"))

# Recall@K 계산
video_mat = torch.nn.functional.normalize(video_mat, dim=-1)
text_mat = torch.nn.functional.normalize(text_mat, dim=-1)

video_mat = video_mat.to(torch.float32)
text_mat = text_mat.to(torch.float32)

sim = video_mat @ text_mat.T

def recall_at_k(sim, k=1):
    ranks = sim.argsort(dim=1, descending=True)
    topk = ranks[:, :k]
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    hits = (topk == correct).any(dim=1).float()
    return hits.mean().item()

print("\n Retrieval 성능 (16 random frames)")
for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k(sim, k):.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
16000 frames resizing using 12 workers...
100%|██████████| 16000/16000 [01:39<00:00, 160.19it/s]
모든 프레임 리사이즈 완료 (224x224).
총 비디오 개수: 1000개
Extracting video embeddings (16 random frames): 100%|██████████| 1000/1000 [01:31<00:00, 10.93it/s]
비디오 임베딩 완료 (1000개)
Extracting text embeddings (mean pooled): 100%|██████████| 1000/1000 [00:11<00:00, 85.22it/s]

Retrieval 성능 (16 random frames)
Recall@1: 0.2860
Recall@5: 0.5210
Recall@10: 0.6260

총 소요 시간: 1.72분


In [11]:
from transformers import CLIPProcessor, CLIPModel
import torch, os
from PIL import Image
from tqdm import tqdm
import json
import time
from multiprocessing import Pool, cpu_count
import warnings

warnings.filterwarnings("ignore", message=".*padding.*")

# 모델 로드 (FP16)
device = "cuda" if torch.cuda.is_available() else "cpu"
model_id = "openai/clip-vit-base-patch16"
model = CLIPModel.from_pretrained(model_id).to(device)
processor = CLIPProcessor.from_pretrained(model_id)
model.eval()

# 병렬 프레임 리사이즈
def _resize_one(args):
    in_path, out_path, size = args
    try:
        img = Image.open(in_path).convert("RGB")
        img = img.resize(size)
        img.save(out_path)
    except Exception:
        pass

def resize_frames_fast(input_dir, output_dir, size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    tasks = []
    for vid in sorted(os.listdir(input_dir)):
        in_vid = os.path.join(input_dir, vid)
        out_vid = os.path.join(output_dir, vid)
        os.makedirs(out_vid, exist_ok=True)
        for f in os.listdir(in_vid):
            if f.endswith(".jpg"):
                tasks.append((os.path.join(in_vid, f),
                              os.path.join(out_vid, f),
                              size))
    print(f" {len(tasks)} frames resizing using {cpu_count()} workers...")
    with Pool(cpu_count()) as p:
        list(tqdm(p.imap_unordered(_resize_one, tasks), total=len(tasks)))
    print(" 모든 프레임 리사이즈 완료 (224x224).")

# 32프레임 랜덤 샘플링 버전 경로
base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL"
original_root = os.path.join(base_dir, "Test_1kA_random32")
resized_root = os.path.join(base_dir, "Test_1kA_random32_224")
resize_frames_fast(original_root, resized_root)

# 비디오 임베딩 추출 (32프레임)
def extract_video_embedding(frame_dir, num_frames=32, batch_size=8):
    frame_files = sorted(os.listdir(frame_dir))[:num_frames]
    images = [Image.open(os.path.join(frame_dir, f)).convert("RGB") for f in frame_files]
    embeds = []
    with torch.no_grad():
        for i in range(0, len(images), batch_size):
            batch = images[i:i+batch_size]
            inputs = processor(images=batch, return_tensors="pt").to(device)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                feats = model.get_image_features(**inputs)
            embeds.append(feats)
    embeds = torch.cat(embeds, dim=0)
    return embeds.mean(dim=0).cpu()

# 텍스트 임베딩 (mean pooling)
def get_text_embeddings(caption_dict, video_ids):
    all_embeds = []
    with torch.no_grad(), torch.autocast(device_type="cuda", dtype=torch.float16):
        for vid in tqdm(video_ids, desc="Extracting text embeddings (mean pooled)"):
            caps = caption_dict[vid]["captions"]
            inputs = processor(text=caps, return_tensors="pt", padding=True, truncation=True).to(device)
            feats = model.get_text_features(**inputs)
            feats = torch.nn.functional.normalize(feats, dim=-1)
            mean_feat = feats.mean(dim=0, keepdim=True)
            all_embeds.append(mean_feat.cpu())
    return torch.cat(all_embeds, dim=0)

# 비디오 및 캡션 로드
video_root = resized_root
video_ids = sorted(os.listdir(video_root))

with open(os.path.join(base_dir, "captions_test_1k.json"), "r") as f:
    caption_dict = json.load(f)

print(f"총 비디오 개수: {len(video_ids)}개")

# 비디오 임베딩 추출
start_time = time.time()
video_embeds = []
for vid in tqdm(video_ids, desc="Extracting video embeddings (32 random frames)"):
    path = os.path.join(video_root, vid)
    try:
        video_embeds.append(extract_video_embedding(path))
    except Exception as e:
        print(f" {vid} 처리 중 오류 발생: {e}")

video_mat = torch.stack(video_embeds)
print(f" 비디오 임베딩 완료 ({len(video_embeds)}개)")

# 텍스트 임베딩 추출
text_mat = get_text_embeddings(caption_dict, video_ids)

# 캐시 저장
torch.save(video_mat, os.path.join(base_dir, "clip_video_embeds_random32f.pt"))
torch.save(text_mat, os.path.join(base_dir, "clip_text_embeds_random32f.pt"))

# Recall@K 계산
video_mat = torch.nn.functional.normalize(video_mat, dim=-1)
text_mat = torch.nn.functional.normalize(text_mat, dim=-1)

video_mat = video_mat.to(torch.float32)
text_mat = text_mat.to(torch.float32)

sim = video_mat @ text_mat.T

def recall_at_k(sim, k=1):
    ranks = sim.argsort(dim=1, descending=True)
    topk = ranks[:, :k]
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    hits = (topk == correct).any(dim=1).float()
    return hits.mean().item()

print("\n Retrieval 성능 (32 random frames)")
for k in [1, 5, 10]:
    print(f"Recall@{k}: {recall_at_k(sim, k):.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
32000 frames resizing using 12 workers...
100%|██████████| 32000/32000 [03:28<00:00, 153.68it/s]
모든 프레임 리사이즈 완료 (224x224).
총 비디오 개수: 1000개
Extracting video embeddings (32 random frames): 100%|██████████| 1000/1000 [03:00<00:00,  5.56it/s]
비디오 임베딩 완료 (1000개)
Extracting text embeddings (mean pooled): 100%|██████████| 1000/1000 [00:11<00:00, 87.02it/s]

Retrieval 성능 (32 random frames)
Recall@1: 0.2710
Recall@5: 0.5230
Recall@10: 0.6400

총 소요 시간: 3.19분


In [ ]:
import torch
import os
import pandas as pd
from tqdm import tqdm

# 실험 파일 목록
experiments = {
    "Uniform_8":  ("clip_video_embeds_8f.pt",  "clip_text_embeds_8f.pt"),
    "Uniform_16": ("clip_video_embeds_16f.pt", "clip_text_embeds_16f.pt"),
    "Uniform_32": ("clip_video_embeds_32f.pt", "clip_text_embeds_32f.pt"),
    "Random_8":   ("clip_video_embeds_random8f.pt",  "clip_text_embeds_random8f.pt"),
    "Random_16":  ("clip_video_embeds_random16f.pt", "clip_text_embeds_random16f.pt"),
    "Random_32":  ("clip_video_embeds_random32f.pt", "clip_text_embeds_random32f.pt"),
}

search_dirs = [
    "/content/drive/MyDrive/VIDEO_RETRIEVAL",
    "/content/drive/MyDrive/video/MSRVTT"
]

# 지표 계산 함수
def compute_metrics(video_mat, text_mat):
    video_mat = torch.nn.functional.normalize(video_mat, dim=-1)
    text_mat = torch.nn.functional.normalize(text_mat, dim=-1)
    sim = video_mat @ text_mat.T

    ranks = sim.argsort(dim=1, descending=True)
    correct = torch.arange(sim.size(0)).unsqueeze(1)
    pos = (ranks == correct).nonzero(as_tuple=False)[:, 1]

    def recall_at_k(k):
        topk = ranks[:, :k]
        hits = (topk == correct).any(dim=1).float()
        return hits.mean().item()

    R1, R5, R10 = recall_at_k(1), recall_at_k(5), recall_at_k(10)
    MedR = pos.median().item()
    MnR = pos.float().mean().item()
    return R1, R5, R10, MedR, MnR

# 파일 탐색 + 결과 계산
results = []

for name, (vfile, tfile) in tqdm(experiments.items(), desc="Evaluating experiments"):
    vpath, tpath = None, None
    for d in search_dirs:
        if vpath is None and os.path.exists(os.path.join(d, vfile)):
            vpath = os.path.join(d, vfile)
        if tpath is None and os.path.exists(os.path.join(d, tfile)):
            tpath = os.path.join(d, tfile)

    if vpath is None or tpath is None:
        print(f"{name}: 파일을 두 경로 어디에서도 찾을 수 없음.")
        continue

    video_mat = torch.load(vpath, map_location="cpu").to(torch.float32)
    text_mat = torch.load(tpath, map_location="cpu").to(torch.float32)

    R1, R5, R10, MedR, MnR = compute_metrics(video_mat, text_mat)
    results.append({
        "Setting": name,
        "R@1": round(R1, 4),
        "R@5": round(R5, 4),
        "R@10": round(R10, 4),
        "MedR": round(MedR, 1),
        "MnR": round(MnR, 1)
    })

order = ["Uniform_8", "Uniform_16", "Uniform_32", "Random_8", "Random_16", "Random_32"]
df = pd.DataFrame(results)
df["Setting"] = pd.Categorical(df["Setting"], categories=order, ordered=True)
df = df.sort_values("Setting")

# 결과표 출력
print("\n Retrieval Evaluation Summary (Recall@K + MedR + MnR)\n")
print(df.to_string(index=False))


Evaluating experiments: 100%|██████████| 6/6 [00:00<00:00, 35.05it/s]


📊 Retrieval Evaluation Summary (Recall@K + MedR + MnR)

   Setting   R@1   R@5  R@10  MedR  MnR
 Uniform_8 0.264 0.510 0.629     4 34.8
Uniform_16 0.273 0.526 0.638     4 34.6
Uniform_32 0.280 0.522 0.641     4 34.3
  Random_8 0.271 0.519 0.628     4 36.2
 Random_16 0.286 0.521 0.626     4 36.2
 Random_32 0.271 0.523 0.640     4 34.6


In [3]:
!pip install av tqdm > /dev/null

import av
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import cv2

train_videos_dir = "/content/drive/MyDrive/video/MSRVTT/TrainVideo"
val_videos_dir   = "/content/drive/MyDrive/video/MSRVTT/ValVideo"

train_frames_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Trainframes16"
val_frames_dir   = "/content/drive/MyDrive/VIDEO_RETRIEVAL/Valframes16"

os.makedirs(train_frames_dir, exist_ok=True)
os.makedirs(val_frames_dir, exist_ok=True)

def extract_uniform_frames_fast(video_path, save_dir, num_frames=16):
    try:
        container = av.open(video_path)
        stream = container.streams.video[0]
        total_frames = stream.frames
        if not total_frames or total_frames < num_frames:
            return False

        # 균등 인덱스 계산
        indices = np.linspace(0, total_frames - 1, num_frames, dtype=int)
        os.makedirs(save_dir, exist_ok=True)

        frames = []
        for i, frame in enumerate(container.decode(stream)):
            if i > indices[-1]:
                break
            if i in indices:
                img = frame.to_ndarray(format="bgr24")
                out_path = os.path.join(save_dir, f"{len(frames):03d}.jpg")
                cv2.imwrite(out_path, img)
                frames.append(out_path)
        container.close()
        return True
    except Exception as e:
        return False

def process_videos_parallel(input_dir, output_dir, num_frames=16, max_workers=8):
    video_files = [f for f in os.listdir(input_dir)
                   if f.lower().endswith((".mp4"))]
    print(f"총 {len(video_files)}개의 비디오 처리 중... (병렬 {max_workers} threads)")

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for vfile in video_files:
            video_path = os.path.join(input_dir, vfile)
            vid_name = os.path.splitext(vfile)[0]
            save_dir = os.path.join(output_dir, vid_name)
            futures.append(executor.submit(extract_uniform_frames_fast,
                                           video_path, save_dir, num_frames))

        for f in tqdm(as_completed(futures), total=len(futures)):
            _ = f.result()


print(" Train 세트 16프레임 Uniform Sampling 시작")
process_videos_parallel(train_videos_dir, train_frames_dir, num_frames=16, max_workers=8)

print("\n Validation 세트 16프레임 Uniform Sampling 시작")
process_videos_parallel(val_videos_dir, val_frames_dir, num_frames=16, max_workers=8)

Train 세트 16프레임 Uniform Sampling 시작
총 6513개의 비디오 처리 중... (병렬 8 threads)
100%|██████████| 6513/6513 [20:46<00:00,  5.23it/s]

Validation 세트 16프레임 Uniform Sampling 시작
총 497개의 비디오 처리 중... (병렬 8 threads)
100%|██████████| 497/497 [01:02<00:00,  8.01it/s]


In [4]:
import os, shutil, random, numpy as np, av, cv2
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

random.seed(42); np.random.seed(42)


SRC = {
    "Train": "/content/drive/MyDrive/video/MSRVTT/TrainVideo",
    "Val":   "/content/drive/MyDrive/video/MSRVTT/ValVideo",
}
TMP = {k: f"/content/MSRVTT_{k}_TMP" for k in SRC}

for k in SRC:
    if not os.path.exists(TMP[k]):
        shutil.copytree(SRC[k], TMP[k], dirs_exist_ok=True)

DST_LOCAL = {
    "Train": {8:"/content/Trainframes_random8",16:"/content/Trainframes_random16",32:"/content/Trainframes_random32"},
    "Val":   {8:"/content/Valframes_random8",  16:"/content/Valframes_random16",  32:"/content/Valframes_random32"},
}
for split in DST_LOCAL:
    for n, p in DST_LOCAL[split].items():
        os.makedirs(p, exist_ok=True)


def list_videos_recursive(root):
    vids = []
    for r, _, fs in os.walk(root):
        for f in fs:
            if f.lower().endswith((".mp4",".avi",".mov",".mkv")):
                vids.append(os.path.join(r, f))
    return vids

def choose_indices(total, k):
    if total <= k: return np.arange(total, dtype=int)
    return np.sort(np.random.choice(total, k, replace=False))

def already_done(save_root, expected_frames=1):
    """이미 프레임 추출된 폴더인지 확인"""
    if not os.path.exists(save_root):
        return False
    imgs = [f for f in os.listdir(save_root) if f.endswith(".jpg")]
    return len(imgs) >= expected_frames

def extract_random_single_pass(video_path, save_root, k=16):
    """랜덤 인덱스를 미리 정렬해 한 번만 디코딩"""
    # 이미 추출된 폴더면 건너뛰기
    if already_done(save_root, expected_frames=k//2):
        return "skip"

    try:
        container = av.open(video_path)
        stream = container.streams.video[0]
        total = stream.frames
        os.makedirs(save_root, exist_ok=True)

        if total and total > 0:
            idx = choose_indices(total, k)
            need = set(idx.tolist())
            max_need = idx[-1]
            saved = 0
            for i, frame in enumerate(container.decode(stream)):
                if i > max_need and saved >= len(idx):
                    break
                if i in need:
                    img = frame.to_ndarray(format="bgr24")
                    cv2.imwrite(os.path.join(save_root, f"frame_{saved}.jpg"), img)
                    saved += 1
            container.close()
            return "ok" if saved > 0 else "fail"

        # total 정보 없는 경우
        frames = [frm for frm in container.decode(stream)]
        total = len(frames)
        if total == 0:
            container.close()
            return "fail"
        idx = choose_indices(total, k)
        for j, i_sel in enumerate(idx):
            img = frames[i_sel].to_ndarray(format="bgr24")
            cv2.imwrite(os.path.join(save_root, f"frame_{j}.jpg"), img)
        container.close()
        return "ok"
    except Exception as e:
        return f"error:{e}"


def process_split(split, n_frames_list=(8,16,32), max_workers=8):
    in_dir = TMP[split]
    files = list_videos_recursive(in_dir)
    total_jobs = len(files) * len(n_frames_list)
    print(f"[{split}] 총 비디오 {len(files)}개, 작업 {total_jobs}개")

    ok = skip = fail = 0
    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = []
        for path in files:
            name = os.path.splitext(os.path.basename(path))[0]
            for k in n_frames_list:
                out_dir = os.path.join(DST_LOCAL[split][k], name)
                futs.append(ex.submit(extract_random_single_pass, path, out_dir, k))

        pbar = tqdm(total=len(futs),
                    desc=f"{split} random (resume mode)",
                    leave=False, position=0, ncols=100)

        for fut in as_completed(futs):
            res = fut.result()
            if res == "ok": ok += 1
            elif res == "skip": skip += 1
            else: fail += 1
            pbar.update(1)

        pbar.close()

    print(f"[{split}] 완료: 성공 {ok}, 건너뜀 {skip}, 실패 {fail}")


process_split("Train", (8,16,32), max_workers=8)
process_split("Val",   (8,16,32), max_workers=8)

# Drive로 동기화
DST_DRIVE = "/content/drive/MyDrive/VIDEO_RETRIEVAL"
for split in DST_LOCAL:
    for n, local_dir in DST_LOCAL[split].items():
        dst = os.path.join(DST_DRIVE, os.path.basename(local_dir))
        shutil.copytree(local_dir, dst, dirs_exist_ok=True)

print(" 랜덤 샘플링(단일 패스, 스킵 모드) 완료. 로컬→Drive 동기화까지 끝.")



[Train] 총 비디오 6513개, 작업 19539개
[Train] 완료: 성공 0, 건너뜀 19539, 실패 0
[Val] 총 비디오 497개, 작업 1491개
[Val] 완료: 성공 0, 건너뜀 1491, 실패 0
랜덤 샘플링(단일 패스, 스킵 모드) 완료. 로컬→Drive 동기화까지 끝.


In [5]:
import shutil

folders = [
    "/content/Trainframes_random8",
    "/content/Trainframes_random16",
    "/content/Trainframes_random32"
]

# 각각 zip으로 묶기
for f in folders:
    if not os.path.exists(f):
        print(f" Not found: {f}")
        continue

    zip_name = f + ".zip"
    print(f" Compressing {f} → {zip_name}")
    shutil.make_archive(f, 'zip', f)
    print("Done.")


Compressing /content/Trainframes_random8 → /content/Trainframes_random8.zip
Done.
Compressing /content/Trainframes_random16 → /content/Trainframes_random16.zip
Done.
Compressing /content/Trainframes_random32 → /content/Trainframes_random32.zip
Done.


In [11]:
import os, zipfile

base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL/random"
zip_files = [
    "Trainframes_random8.zip",
    "Trainframes_random16.zip",
    "Trainframes_random32.zip"
]

for zf in zip_files:
    zip_path = os.path.join(base_dir, zf)
    extract_dir = os.path.join(base_dir, zf.replace(".zip", ""))
    os.makedirs(extract_dir, exist_ok=True)

    print(f" {zf} 압축 해제 중 → {extract_dir}")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f" 완료: {extract_dir}\n")


Trainframes_random8.zip 압축 해제 중 → /content/drive/MyDrive/VIDEO_RETRIEVAL/random/Trainframes_random8
완료: /content/drive/MyDrive/VIDEO_RETRIEVAL/random/Trainframes_random8

Trainframes_random16.zip 압축 해제 중 → /content/drive/MyDrive/VIDEO_RETRIEVAL/random/Trainframes_random16
완료: /content/drive/MyDrive/VIDEO_RETRIEVAL/random/Trainframes_random16

Trainframes_random32.zip 압축 해제 중 → /content/drive/MyDrive/VIDEO_RETRIEVAL/random/Trainframes_random32
완료: /content/drive/MyDrive/VIDEO_RETRIEVAL/random/Trainframes_random32


In [10]:
import torch
import os

base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL"

video_feats_train_path = os.path.join(base_dir, "clip_video_embeds_train_8f.pt")
video_feats_val_path   = os.path.join(base_dir, "clip_video_embeds_val_8f.pt")
text_feats_train_path  = os.path.join(base_dir, "clip_text_embeds_train_8f.pt")
text_feats_val_path    = os.path.join(base_dir, "clip_text_embeds_val_8f.pt")

paths = {
    "video_feats_train": video_feats_train_path,
    "video_feats_val": video_feats_val_path,
    "text_feats_train": text_feats_train_path,
    "text_feats_val": text_feats_val_path,
}

for name, path in paths.items():
    if not os.path.exists(path):
        print(f" {name}: 파일이 존재하지 않음 ({path})")
        continue

    data = torch.load(path, map_location="cpu")
    if isinstance(data, torch.Tensor):
        print(f" {name}: {tuple(data.shape)}")
    elif isinstance(data, dict):
        print(f" {name}: dict 구조 ({len(data.keys())} keys)")
        for k, v in data.items():
            if isinstance(v, torch.Tensor):
                print(f"    └─ {k}: {tuple(v.shape)}")
    elif isinstance(data, list):
        print(f" {name}: list 구조 (길이 {len(data)})")
        if isinstance(data[0], torch.Tensor):
            print(f"    └─ 첫 번째 요소 크기: {tuple(data[0].shape)}")
    else:
        print(f" {name}: 알 수 없는 타입 {type(data)}")


video_feats_train: list 구조 (길이 6513)
    └─ 첫 번째 요소 크기: (8, 512)
video_feats_val: list 구조 (길이 497)
    └─ 첫 번째 요소 크기: (8, 512)
text_feats_train: list 구조 (길이 6513)
    └─ 첫 번째 요소 크기: (512,)
text_feats_val: list 구조 (길이 497)
    └─ 첫 번째 요소 크기: (512,)


In [7]:
import torch
import os

base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL"

video_feats_train_path = os.path.join(base_dir, "clip_video_embeds_train_16f.pt")
video_feats_val_path   = os.path.join(base_dir, "clip_video_embeds_val_16f.pt")
text_feats_train_path  = os.path.join(base_dir, "clip_text_embeds_train_16f.pt")
text_feats_val_path    = os.path.join(base_dir, "clip_text_embeds_val_16f.pt")

paths = {
    "video_feats_train": video_feats_train_path,
    "video_feats_val": video_feats_val_path,
    "text_feats_train": text_feats_train_path,
    "text_feats_val": text_feats_val_path,
}

for name, path in paths.items():
    if not os.path.exists(path):
        print(f" {name}: 파일이 존재하지 않음 ({path})")
        continue

    data = torch.load(path, map_location="cpu")
    if isinstance(data, torch.Tensor):
        print(f" {name}: {tuple(data.shape)}")
    elif isinstance(data, dict):
        print(f" {name}: dict 구조 ({len(data.keys())} keys)")
        for k, v in data.items():
            if isinstance(v, torch.Tensor):
                print(f"    └─ {k}: {tuple(v.shape)}")
    elif isinstance(data, list):
        print(f" {name}: list 구조 (길이 {len(data)})")
        if isinstance(data[0], torch.Tensor):
            print(f"    └─ 첫 번째 요소 크기: {tuple(data[0].shape)}")
    else:
        print(f" {name}: 알 수 없는 타입 {type(data)}")


video_feats_train: list 구조 (길이 6513)
    └─ 첫 번째 요소 크기: (16, 512)
video_feats_val: list 구조 (길이 497)
    └─ 첫 번째 요소 크기: (16, 512)
text_feats_train: list 구조 (길이 6513)
    └─ 첫 번째 요소 크기: (512,)
text_feats_val: list 구조 (길이 497)
    └─ 첫 번째 요소 크기: (512,)


In [6]:
import torch
import os

base_dir = "/content/drive/MyDrive/VIDEO_RETRIEVAL"

video_feats_train_path = os.path.join(base_dir, "clip_video_embeds_train_32f.pt")
video_feats_val_path   = os.path.join(base_dir, "clip_video_embeds_val_32f.pt")
text_feats_train_path  = os.path.join(base_dir, "clip_text_embeds_train_32f.pt")
text_feats_val_path    = os.path.join(base_dir, "clip_text_embeds_val_32f.pt")

paths = {
    "video_feats_train": video_feats_train_path,
    "video_feats_val": video_feats_val_path,
    "text_feats_train": text_feats_train_path,
    "text_feats_val": text_feats_val_path,
}

for name, path in paths.items():
    if not os.path.exists(path):
        print(f" {name}: 파일이 존재하지 않음 ({path})")
        continue

    data = torch.load(path, map_location="cpu")
    if isinstance(data, torch.Tensor):
        print(f" {name}: {tuple(data.shape)}")
    elif isinstance(data, dict):
        print(f" {name}: dict 구조 ({len(data.keys())} keys)")
        for k, v in data.items():
            if isinstance(v, torch.Tensor):
                print(f"    └─ {k}: {tuple(v.shape)}")
    elif isinstance(data, list):
        print(f" {name}: list 구조 (길이 {len(data)})")
        if isinstance(data[0], torch.Tensor):
            print(f"    └─ 첫 번째 요소 크기: {tuple(data[0].shape)}")
    else:
        print(f" {name}: 알 수 없는 타입 {type(data)}")


video_feats_train: list 구조 (길이 6513)
    └─ 첫 번째 요소 크기: (32, 512)
video_feats_val: list 구조 (길이 497)
    └─ 첫 번째 요소 크기: (32, 512)
text_feats_train: list 구조 (길이 6513)
    └─ 첫 번째 요소 크기: (512,)
text_feats_val: list 구조 (길이 497)
    └─ 첫 번째 요소 크기: (512,)
